<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_overlap40_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_40.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,3,cracking sensations in my joints
2,4,purple sploches around waist
3,5,fast and slow heartbeat
4,5,increased heart rate


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'my',
 'is',
 'myalgia',
 'lower',
 'fatigue',
 'rt',
 'loss',
 'by',
 'limb',
 'sodium',
 'cramp',
 'weakness',
 'at',
 'disease',
 'stomach',
 'that',
 'or',
 'back',
 'as',
 '-',
 'depression',
 'from',
 'patients',
 'i',
 'this',
 'not',
 'cramps',
 'upper',
 'joint',
 'gas',
 'drug',
 'memory',
 'leg',
 'you',
 'mg',
 'all',
 'pains',
 'legs',
 'knee',
 'be',
 'are',
 'headache',
 'shoulder',
 'unable',
 'group',
 'study',
 'arthralgia',
 'gastrointestinal',
 'treatment',
 'neck',
 'excessive',
 'foot',
 'arthritis',
 'it',
 'aches',
 'after',
 'an',
 'amp',
 'hip',
 'have',
 'voltaren',
 'like',
 'numbness',
 'insomnia',
 'arthrotec',
 'p',
 'no',
 'release',
 'bleeding',
 'both',
 'gluten',
 '/',
 'nausea',
 'time',
 'asthenia',
 'can',
 'increased',

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.847866,3.487731,0.442138,32:37


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.514899,3.427935,0.447205,32:22


In [25]:
learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap40_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_bigtweet_askpatient_mixed_overlap40_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.028854,3.725535,0.317160,01:11


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.941269,3.049603,0.469822,01:28


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.397470,2.700058,0.531361,03:39


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.104843,2.334436,0.607101,03:35
1,1.705963,2.195746,0.628402,03:13


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.523691,2.001046,0.656805,03:01
1,1.316889,1.887292,0.676923,02:55


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.197496,1.779742,0.688757,03:05
1,1.108467,1.688480,0.707692,03:00


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.995348,1.601994,0.724260,02:58
1,0.921214,1.545022,0.730178,02:54


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.813956,1.489066,0.744379,03:06
1,0.716107,1.446226,0.739645,03:15


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.675624,1.404308,0.763314,03:22
1,0.645397,1.395528,0.765680,02:59
2,0.543138,1.346788,0.769231,03:19
3,0.462294,1.349427,0.776331,03:23


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.486377,1.377117,0.773965,03:14
1,0.551098,1.372133,0.765680,03:03
2,0.470658,1.363899,0.786982,02:58
3,0.366066,1.321194,0.788166,03:12


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.359817,1.318236,0.779882,03:15
1,0.384018,1.330479,0.794083,03:05
2,0.331408,1.294020,0.796450,03:13
3,0.304500,1.299855,0.800000,03:07


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.284028,1.407340,0.792899,03:20
1,0.344618,1.247649,0.796450,03:19
2,0.285218,1.303871,0.800000,02:55
3,0.263149,1.309771,0.804734,02:57


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.258395,1.361946,0.800000,03:14
1,0.321182,1.365519,0.796450,03:05
2,0.237267,1.362995,0.805917,03:15
3,0.231942,1.350136,0.808284,03:09


In [39]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [40]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [41]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [42]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.246106,1.424467,0.803550,03:02


In [44]:
# save the best model

learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap40')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

1018
1018
[0, 343, 520, 689, 5, 1026, 1026, 8, 8, 8, 8, 8, 8, 511, 10, 368, 331, 379, 21, 555, 27, 644, 545, 29, 260, 289, 30, 31, 31, 32, 34, 289, 37, 37, 37, 767, 348, 41, 255, 996, 996, 49, 49, 325, 49, 49, 49, 49, 49, 49, 100, 100, 100, 100, 348, 1026, 53, 61, 70, 71, 71, 71, 491, 76, 76, 76, 318, 182, 614, 1026, 1026, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 289, 293, 289, 228, 79, 79, 80, 80, 80, 80, 80, 80, 80, 80, 82, 88, 89, 137, 391, 90, 91, 92, 469, 94, 94, 182, 94, 94, 94, 94, 96, 620, 291, 683, 98, 794, 337, 100, 100, 337, 100, 100, 100, 100, 15, 100, 291, 236, 107, 108, 108, 108, 209, 43, 289, 111, 111, 111, 289, 348, 111, 281, 289, 289, 730, 117, 973, 120, 122, 90, 1026, 125, 125, 125, 125, 125, 125, 125, 125, 1026, 125, 127, 741, 289, 289, 172, 132, 627, 447, 447, 144, 481, 827, 151, 282, 37, 344, 34, 158, 163, 165, 166, 166, 166, 168, 170, 172, 614, 172, 172, 172, 172, 172, 742, 182, 18

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

586
0.5756385068762279
